优化矩阵乘法

矩阵乘法是计算密集型运算，为了取得良好的CPU性能，有两个重要的优化
+ 提高内存访问的Cache命中率，需要将原始内存访问模式转换为适合缓存策略的模式，提高局部性
+ SIMD（单指令多数据），向量处理单元，在每个循环中处理一小批数据而不是处理单个值，需要将循环体中的数据访问模式转换为统一模式，以便LLVM后端可以将其降低到SIMD

In [2]:
import tvm
import tvm.testing
from tvm import te
import numpy
import timeit

# 矩阵的大小
# (M, K) x (K, N)
# 可尝试不同的 shape，TVM 优化的性能有时比 numpy + MKL 更好
M = 1024
K = 1024
N = 1024


# TVM 默认张量数据类型
dtype = "float32"

# 你可能想调整 target 使其和你的任何 CPU 向量扩展匹配
# 例如，如果你为 SIMD 用的是 Intel AVX2（高级向量扩展）ISA，把下面这行换成 `llvm -mcpu=core-avx2` 可以取得最佳性能（或者你所用 CPU 的具体类型）
# 记住你用的是 llvm, 可以用 `llc --version` 命令来获取 CPU 类型，也可以查看 `/proc/cpuinfo` 来获取你处理器支持的更多扩展

target = tvm.target.Target(target="llvm", host="llvm")
dev = tvm.device(target.kind.name, 0)

# 为测试随机生成的张量
a = tvm.nd.array(numpy.random.rand(M, K).astype(dtype), dev)
b = tvm.nd.array(numpy.random.rand(K, N).astype(dtype), dev)

# 重复执行矩阵乘法以获得默认 numpy 实现的性能基线
np_repeat = 100
np_running_time = timeit.timeit(
    setup="import numpy\n"
    "M = " + str(M) + "\n"
    "K = " + str(K) + "\n"
    "N = " + str(N) + "\n"
    'dtype = "float32"\n'
    "a = numpy.random.rand(M, K).astype(dtype)\n"
    "b = numpy.random.rand(K, N).astype(dtype)\n",
    stmt="answer = numpy.dot(a, b)",
    number=np_repeat,
)
print("Numpy running time: %f" % (np_running_time / np_repeat))

answer = numpy.dot(a.numpy(), b.numpy())

[16:56:58] /home/patrick/Code/tvm/src/runtime/logging.cc:307: TVM_LOG_DEBUG enables VLOG statements in 'ir/transform.cc' up to level 1
[16:56:58] /home/patrick/Code/tvm/src/runtime/logging.cc:307: TVM_LOG_DEBUG enables VLOG statements in 'relay/ir/transform.cc' up to level 1


Numpy running time: 0.006408


用TVM TE编写一个基本的矩阵乘法，并验证它是否产生与numpy实现相同的结果，在探索性能

In [3]:
# 用 TE 的 TVM 矩阵乘法
k = te.reduce_axis((0, K), "k")
A = te.placeholder((M, K), name="A")
B = te.placeholder((K, N), name="B")
C = te.compute((M, N), lambda x, y: te.sum(A[x, k] * B[k, y], axis=k), name="C")

# 默认 schedule
s = te.create_schedule(C.op)
func = tvm.build(s, [A, B, C], target=target, name="mmult")

c = tvm.nd.array(numpy.zeros((M, N), dtype=dtype), dev)
func(a, b, c)
tvm.testing.assert_allclose(c.numpy(), answer, rtol=1e-5)

def evaluate_operation(s, vars, target, name, optimization, log):
    func = tvm.build(s, [A, B, C], target=target, name="mmult")
    assert func

    c = tvm.nd.array(numpy.zeros((M, N), dtype=dtype), dev)
    func(a, b, c)
    tvm.testing.assert_allclose(c.numpy(), answer, rtol=1e-5)

    evaluator = func.time_evaluator(func.entry_name, dev, number=10)
    mean_time = evaluator(a, b, c).mean
    print("%s: %f" % (optimization, mean_time))
    log.append((optimization, mean_time))

log = []

evaluate_operation(s, [A, B, C], target=target, name="mmult", optimization="none", log=log)

[16:57:00] /home/patrick/Code/tvm/src/ir/transform.cc:440: Running pass tir.InjectPrefetch
[16:57:00] /home/patrick/Code/tvm/src/ir/transform.cc:440: Running pass tir.TextureFlatten
[16:57:00] /home/patrick/Code/tvm/src/ir/transform.cc:440: Running pass tir.StorageFlatten
[16:57:00] /home/patrick/Code/tvm/src/ir/transform.cc:440: Running pass tir.BufferShapeLegalize
[16:57:00] /home/patrick/Code/tvm/src/ir/transform.cc:440: Running pass tir.BufferStrideLegalize
[16:57:00] /home/patrick/Code/tvm/src/ir/transform.cc:440: Running pass tir.ThreadScopePropagate
[16:57:00] /home/patrick/Code/tvm/src/ir/transform.cc:440: Running pass tir.BufferBindUnwrapper
[16:57:00] /home/patrick/Code/tvm/src/ir/transform.cc:440: Running pass tir.ApplyLayoutTransforms
[16:57:00] /home/patrick/Code/tvm/src/ir/transform.cc:440: Running pass tir.StorageFlattener
[16:57:00] /home/patrick/Code/tvm/src/ir/transform.cc:440: Running pass tir.AssertSimplifier
[16:57:00] /home/patrick/Code/tvm/src/ir/transform.cc:440

none: 2.532850


查看用TVM底层函数的算子和默认调度的中间表示

In [4]:
print(tvm.lower(s, [A, B, C], simple_mode=True))

# from tvm.script import ir as I
# from tvm.script import tir as T

@I.ir_module
class Module:
    @T.prim_func
    def main(A: T.Buffer((1024, 1024), "float32"), B: T.Buffer((1024, 1024), "float32"), C: T.Buffer((1024, 1024), "float32")):
        T.func_attr({"from_legacy_te_schedule": True, "global_symbol": "main", "tir.noalias": True})
        for x, y in T.grid(1024, 1024):
            C_1 = T.Buffer((1048576,), data=C.data)
            C_1[x * 1024 + y] = T.float32(0)
            for k in range(1024):
                cse_var_2: T.int32 = x * 1024
                cse_var_1: T.int32 = cse_var_2 + y
                A_1 = T.Buffer((1048576,), data=A.data)
                B_1 = T.Buffer((1048576,), data=B.data)
                C_1[cse_var_1] = C_1[cse_var_1] + A_1[cse_var_2 + k] * B_1[k * 1024 + y]


[16:57:33] /home/patrick/Code/tvm/src/ir/transform.cc:440: Running pass tir.InjectPrefetch
[16:57:33] /home/patrick/Code/tvm/src/ir/transform.cc:440: Running pass tir.TextureFlatten
[16:57:33] /home/patrick/Code/tvm/src/ir/transform.cc:440: Running pass tir.StorageFlatten
[16:57:33] /home/patrick/Code/tvm/src/ir/transform.cc:440: Running pass tir.BufferShapeLegalize
[16:57:33] /home/patrick/Code/tvm/src/ir/transform.cc:440: Running pass tir.BufferStrideLegalize
[16:57:33] /home/patrick/Code/tvm/src/ir/transform.cc:440: Running pass tir.ThreadScopePropagate
[16:57:33] /home/patrick/Code/tvm/src/ir/transform.cc:440: Running pass tir.BufferBindUnwrapper
[16:57:33] /home/patrick/Code/tvm/src/ir/transform.cc:440: Running pass tir.ApplyLayoutTransforms
[16:57:33] /home/patrick/Code/tvm/src/ir/transform.cc:440: Running pass tir.StorageFlattener
[16:57:33] /home/patrick/Code/tvm/src/ir/transform.cc:440: Running pass tir.AssertSimplifier
[16:57:33] /home/patrick/Code/tvm/src/ir/transform.cc:440

优化一，块操作

可以在其中构造内存访问，使块内部是具有高内存局部性的小邻域

首先为C操作创建一个默认schedule，然后用指定的块因子对其应用tile调度原语，调度原语返回向量[x_outer, y_outer, x_inner, y_inner]，表示从最外层到最内层的结果循环的顺序，然后得到操作输出的归约轴并用因子4对其执行拆分操作

既然操作已经块级化了，可对计算进行重新排序，将归约操作放到计算的最外层循环中，保证块数据保留在缓存中。完成schedule后，就可以构建和测试与原始schedule相比的性能

In [5]:
bn = 32

# 通过循环切分实现块级化
xo, yo, xi, yi = s[C].tile(C.op.axis[0], C.op.axis[1], bn, bn)
(k,) = s[C].op.reduce_axis
ko, ki = s[C].split(k, factor=4)

# 将归约域提升到块循环外
s[C].reorder(xo, yo, ko, ki, xi, yi)

evaluate_operation(s, [A, B, C], target=target, name="mmult", optimization="blocking", log=log)

[20:42:00] /home/patrick/Code/tvm/src/ir/transform.cc:440: Running pass tir.InjectPrefetch
[20:42:00] /home/patrick/Code/tvm/src/ir/transform.cc:440: Running pass tir.TextureFlatten
[20:42:00] /home/patrick/Code/tvm/src/ir/transform.cc:440: Running pass tir.StorageFlatten
[20:42:00] /home/patrick/Code/tvm/src/ir/transform.cc:440: Running pass tir.BufferShapeLegalize
[20:42:00] /home/patrick/Code/tvm/src/ir/transform.cc:440: Running pass tir.BufferStrideLegalize
[20:42:00] /home/patrick/Code/tvm/src/ir/transform.cc:440: Running pass tir.ThreadScopePropagate
[20:42:00] /home/patrick/Code/tvm/src/ir/transform.cc:440: Running pass tir.BufferBindUnwrapper
[20:42:00] /home/patrick/Code/tvm/src/ir/transform.cc:440: Running pass tir.ApplyLayoutTransforms
[20:42:00] /home/patrick/Code/tvm/src/ir/transform.cc:440: Running pass tir.StorageFlattener
[20:42:00] /home/patrick/Code/tvm/src/ir/transform.cc:440: Running pass tir.AssertSimplifier
[20:42:00] /home/patrick/Code/tvm/src/ir/transform.cc:440

blocking: 0.167352


In [6]:
print(tvm.lower(s, [A, B, C], simple_mode=True))

# from tvm.script import ir as I
# from tvm.script import tir as T

@I.ir_module
class Module:
    @T.prim_func
    def main(A: T.Buffer((1024, 1024), "float32"), B: T.Buffer((1024, 1024), "float32"), C: T.Buffer((1024, 1024), "float32")):
        T.func_attr({"from_legacy_te_schedule": True, "global_symbol": "main", "tir.noalias": True})
        for x_outer, y_outer in T.grid(32, 32):
            C_1 = T.Buffer((1048576,), data=C.data)
            for x_inner_init, y_inner_init in T.grid(32, 32):
                C_1[x_outer * 32768 + x_inner_init * 1024 + y_outer * 32 + y_inner_init] = T.float32(0)
            for k_outer, k_inner, x_inner, y_inner in T.grid(256, 4, 32, 32):
                cse_var_3: T.int32 = y_outer * 32
                cse_var_2: T.int32 = x_outer * 32768 + x_inner * 1024
                cse_var_1: T.int32 = cse_var_2 + cse_var_3 + y_inner
                A_1 = T.Buffer((1048576,), data=A.data)
                B_1 = T.Buffer((1048576,), data=B.data)
              

[20:42:20] /home/patrick/Code/tvm/src/ir/transform.cc:440: Running pass tir.InjectPrefetch
[20:42:20] /home/patrick/Code/tvm/src/ir/transform.cc:440: Running pass tir.TextureFlatten
[20:42:20] /home/patrick/Code/tvm/src/ir/transform.cc:440: Running pass tir.StorageFlatten
[20:42:20] /home/patrick/Code/tvm/src/ir/transform.cc:440: Running pass tir.BufferShapeLegalize
[20:42:20] /home/patrick/Code/tvm/src/ir/transform.cc:440: Running pass tir.BufferStrideLegalize
[20:42:20] /home/patrick/Code/tvm/src/ir/transform.cc:440: Running pass tir.ThreadScopePropagate
[20:42:20] /home/patrick/Code/tvm/src/ir/transform.cc:440: Running pass tir.BufferBindUnwrapper
[20:42:20] /home/patrick/Code/tvm/src/ir/transform.cc:440: Running pass tir.ApplyLayoutTransforms
[20:42:20] /home/patrick/Code/tvm/src/ir/transform.cc:440: Running pass tir.StorageFlattener
[20:42:20] /home/patrick/Code/tvm/src/ir/transform.cc:440: Running pass tir.AssertSimplifier
[20:42:20] /home/patrick/Code/tvm/src/ir/transform.cc:440

向量化

另一个重要的优化技巧是向量化，当内存访问模式一致时，编译器可以检测这些模式，并将连续内存传递给SIMD向量处理器，利用TVM中这个硬件特征，可以用vectorize接口来提示编译器这个模式

In [7]:
# Apply the vectorization optimization
s[C].vectorize(yi)

evaluate_operation(s, [A, B, C], target=target, name="mmult", optimization="vectorization", log=log)

# The generalized IR after vectorization
print(tvm.lower(s, [A, B, C], simple_mode=True))

[20:50:58] /home/patrick/Code/tvm/src/ir/transform.cc:440: Running pass tir.InjectPrefetch
[20:50:58] /home/patrick/Code/tvm/src/ir/transform.cc:440: Running pass tir.TextureFlatten
[20:50:58] /home/patrick/Code/tvm/src/ir/transform.cc:440: Running pass tir.StorageFlatten
[20:50:58] /home/patrick/Code/tvm/src/ir/transform.cc:440: Running pass tir.BufferShapeLegalize
[20:50:58] /home/patrick/Code/tvm/src/ir/transform.cc:440: Running pass tir.BufferStrideLegalize
[20:50:58] /home/patrick/Code/tvm/src/ir/transform.cc:440: Running pass tir.ThreadScopePropagate
[20:50:58] /home/patrick/Code/tvm/src/ir/transform.cc:440: Running pass tir.BufferBindUnwrapper
[20:50:58] /home/patrick/Code/tvm/src/ir/transform.cc:440: Running pass tir.ApplyLayoutTransforms
[20:50:58] /home/patrick/Code/tvm/src/ir/transform.cc:440: Running pass tir.StorageFlattener
[20:50:58] /home/patrick/Code/tvm/src/ir/transform.cc:440: Running pass tir.AssertSimplifier
[20:50:58] /home/patrick/Code/tvm/src/ir/transform.cc:440

vectorization: 0.166719
# from tvm.script import ir as I
# from tvm.script import tir as T

@I.ir_module
class Module:
    @T.prim_func
    def main(A: T.Buffer((1024, 1024), "float32"), B: T.Buffer((1024, 1024), "float32"), C: T.Buffer((1024, 1024), "float32")):
        T.func_attr({"from_legacy_te_schedule": True, "global_symbol": "main", "tir.noalias": True})
        for x_outer, y_outer in T.grid(32, 32):
            C_1 = T.Buffer((1048576,), data=C.data)
            for x_inner_init in range(32):
                C_1[x_outer * 32768 + x_inner_init * 1024 + y_outer * 32:x_outer * 32768 + x_inner_init * 1024 + y_outer * 32 + 32] = T.Broadcast(T.float32(0), 32)
            for k_outer, k_inner, x_inner in T.grid(256, 4, 32):
                cse_var_3: T.int32 = y_outer * 32
                cse_var_2: T.int32 = x_outer * 32768 + x_inner * 1024
                cse_var_1: T.int32 = cse_var_2 + cse_var_3
                A_1 = T.Buffer((1048576,), data=A.data)
                B_1 = T.Buff

[20:51:00] /home/patrick/Code/tvm/src/ir/transform.cc:440: Running pass tir.InjectPrefetch
[20:51:00] /home/patrick/Code/tvm/src/ir/transform.cc:440: Running pass tir.TextureFlatten
[20:51:00] /home/patrick/Code/tvm/src/ir/transform.cc:440: Running pass tir.StorageFlatten
[20:51:00] /home/patrick/Code/tvm/src/ir/transform.cc:440: Running pass tir.BufferShapeLegalize
[20:51:00] /home/patrick/Code/tvm/src/ir/transform.cc:440: Running pass tir.BufferStrideLegalize
[20:51:00] /home/patrick/Code/tvm/src/ir/transform.cc:440: Running pass tir.ThreadScopePropagate
[20:51:00] /home/patrick/Code/tvm/src/ir/transform.cc:440: Running pass tir.BufferBindUnwrapper
[20:51:00] /home/patrick/Code/tvm/src/ir/transform.cc:440: Running pass tir.ApplyLayoutTransforms
[20:51:00] /home/patrick/Code/tvm/src/ir/transform.cc:440: Running pass tir.StorageFlattener
[20:51:00] /home/patrick/Code/tvm/src/ir/transform.cc:440: Running pass tir.AssertSimplifier
[20:51:00] /home/patrick/Code/tvm/src/ir/transform.cc:440

循环置换

查看上面的IR，可以看到内部循环行数据被向量化，并且 B 被转换为 PackedB（通过内部循环的 (float32x32)B2 部分可明显看出）。 PackedB 的遍历现在是顺序的。在当前 schedule 中，A 是逐列访问的，这对缓存不利。如果我们改变 ki 和内轴 xi* 的嵌套循环顺序，A 矩阵的访问模式将更利于缓存

In [8]:
s = te.create_schedule(C.op)
xo, yo, xi, yi = s[C].tile(C.op.axis[0], C.op.axis[1], bn, bn)
(k,) = s[C].op.reduce_axis
ko, ki = s[C].split(k, factor=4)

# re-ordering
# 重新排序
s[C].reorder(xo, yo, ko, xi, ki, yi)
s[C].vectorize(yi)

evaluate_operation(
    s, [A, B, C], target=target, name="mmult", optimization="loop permutation", log=log
)

# Again, print the new generalized IR
# 再次打印新生成的 IR
print(tvm.lower(s, [A, B, C], simple_mode=True))

[20:55:44] /home/patrick/Code/tvm/src/ir/transform.cc:440: Running pass tir.InjectPrefetch
[20:55:44] /home/patrick/Code/tvm/src/ir/transform.cc:440: Running pass tir.TextureFlatten
[20:55:44] /home/patrick/Code/tvm/src/ir/transform.cc:440: Running pass tir.StorageFlatten
[20:55:44] /home/patrick/Code/tvm/src/ir/transform.cc:440: Running pass tir.BufferShapeLegalize
[20:55:44] /home/patrick/Code/tvm/src/ir/transform.cc:440: Running pass tir.BufferStrideLegalize
[20:55:44] /home/patrick/Code/tvm/src/ir/transform.cc:440: Running pass tir.ThreadScopePropagate
[20:55:44] /home/patrick/Code/tvm/src/ir/transform.cc:440: Running pass tir.BufferBindUnwrapper
[20:55:44] /home/patrick/Code/tvm/src/ir/transform.cc:440: Running pass tir.ApplyLayoutTransforms
[20:55:44] /home/patrick/Code/tvm/src/ir/transform.cc:440: Running pass tir.StorageFlattener
[20:55:44] /home/patrick/Code/tvm/src/ir/transform.cc:440: Running pass tir.AssertSimplifier
[20:55:44] /home/patrick/Code/tvm/src/ir/transform.cc:440

loop permutation: 0.066699
# from tvm.script import ir as I
# from tvm.script import tir as T

@I.ir_module
class Module:
    @T.prim_func
    def main(A: T.Buffer((1024, 1024), "float32"), B: T.Buffer((1024, 1024), "float32"), C: T.Buffer((1024, 1024), "float32")):
        T.func_attr({"from_legacy_te_schedule": True, "global_symbol": "main", "tir.noalias": True})
        for x_outer, y_outer in T.grid(32, 32):
            C_1 = T.Buffer((1048576,), data=C.data)
            for x_inner_init in range(32):
                C_1[x_outer * 32768 + x_inner_init * 1024 + y_outer * 32:x_outer * 32768 + x_inner_init * 1024 + y_outer * 32 + 32] = T.Broadcast(T.float32(0), 32)
            for k_outer, x_inner, k_inner in T.grid(256, 32, 4):
                cse_var_3: T.int32 = y_outer * 32
                cse_var_2: T.int32 = x_outer * 32768 + x_inner * 1024
                cse_var_1: T.int32 = cse_var_2 + cse_var_3
                A_1 = T.Buffer((1048576,), data=A.data)
                B_1 = T.B

[20:55:45] /home/patrick/Code/tvm/src/ir/transform.cc:440: Running pass tir.InjectPrefetch
[20:55:45] /home/patrick/Code/tvm/src/ir/transform.cc:440: Running pass tir.TextureFlatten
[20:55:45] /home/patrick/Code/tvm/src/ir/transform.cc:440: Running pass tir.StorageFlatten
[20:55:45] /home/patrick/Code/tvm/src/ir/transform.cc:440: Running pass tir.BufferShapeLegalize
[20:55:45] /home/patrick/Code/tvm/src/ir/transform.cc:440: Running pass tir.BufferStrideLegalize
[20:55:45] /home/patrick/Code/tvm/src/ir/transform.cc:440: Running pass tir.ThreadScopePropagate
[20:55:45] /home/patrick/Code/tvm/src/ir/transform.cc:440: Running pass tir.BufferBindUnwrapper
[20:55:45] /home/patrick/Code/tvm/src/ir/transform.cc:440: Running pass tir.ApplyLayoutTransforms
[20:55:45] /home/patrick/Code/tvm/src/ir/transform.cc:440: Running pass tir.StorageFlattener
[20:55:45] /home/patrick/Code/tvm/src/ir/transform.cc:440: Running pass tir.AssertSimplifier
[20:55:45] /home/patrick/Code/tvm/src/ir/transform.cc:440

数组打包

数组打包对数组的存储维度进行重新排序，将某个维度上的连续访问模式转换为展开后的顺序模式

In [9]:
# We have to re-write the algorithm slightly.
# 我们必须稍作改动以重写算法
packedB = te.compute((N / bn, K, bn), lambda x, y, z: B[y, x * bn + z], name="packedB")
C = te.compute(
    (M, N),
    lambda x, y: te.sum(A[x, k] * packedB[y // bn, k, tvm.tir.indexmod(y, bn)], axis=k),
    name="C",
)

s = te.create_schedule(C.op)

xo, yo, xi, yi = s[C].tile(C.op.axis[0], C.op.axis[1], bn, bn)
(k,) = s[C].op.reduce_axis
ko, ki = s[C].split(k, factor=4)

s[C].reorder(xo, yo, ko, xi, ki, yi)
s[C].vectorize(yi)

x, y, z = s[packedB].op.axis
s[packedB].vectorize(z)
s[packedB].parallel(x)

evaluate_operation(s, [A, B, C], target=target, name="mmult", optimization="array packing", log=log)

# Here is the generated IR after array packing.
# 数组打包后生成的 IR
print(tvm.lower(s, [A, B, C], simple_mode=True))

[21:09:29] /home/patrick/Code/tvm/src/ir/transform.cc:440: Running pass tir.InjectPrefetch
[21:09:29] /home/patrick/Code/tvm/src/ir/transform.cc:440: Running pass tir.TextureFlatten
[21:09:29] /home/patrick/Code/tvm/src/ir/transform.cc:440: Running pass tir.StorageFlatten
[21:09:29] /home/patrick/Code/tvm/src/ir/transform.cc:440: Running pass tir.BufferShapeLegalize
[21:09:29] /home/patrick/Code/tvm/src/ir/transform.cc:440: Running pass tir.BufferStrideLegalize
[21:09:29] /home/patrick/Code/tvm/src/ir/transform.cc:440: Running pass tir.ThreadScopePropagate
[21:09:29] /home/patrick/Code/tvm/src/ir/transform.cc:440: Running pass tir.BufferBindUnwrapper
[21:09:29] /home/patrick/Code/tvm/src/ir/transform.cc:440: Running pass tir.ApplyLayoutTransforms
[21:09:29] /home/patrick/Code/tvm/src/ir/transform.cc:440: Running pass tir.StorageFlattener
[21:09:29] /home/patrick/Code/tvm/src/ir/transform.cc:440: Running pass tir.AssertSimplifier
[21:09:29] /home/patrick/Code/tvm/src/ir/transform.cc:440

array packing: 0.063167
# from tvm.script import ir as I
# from tvm.script import tir as T

@I.ir_module
class Module:
    @T.prim_func
    def main(A: T.Buffer((1024, 1024), "float32"), B: T.Buffer((1024, 1024), "float32"), C: T.Buffer((1024, 1024), "float32")):
        T.func_attr({"from_legacy_te_schedule": True, "global_symbol": "main", "tir.noalias": True})
        packedB = T.allocate([32768], "float32x32", "global")
        packedB_1 = T.Buffer((32768,), "float32x32", data=packedB)
        for x in T.parallel(32):
            for y in range(1024):
                B_1 = T.Buffer((1048576,), data=B.data)
                packedB_1[x * 1024 + y] = B_1[y * 1024 + x * 32:y * 1024 + x * 32 + 32]
        for x_outer, y_outer in T.grid(32, 32):
            C_1 = T.Buffer((1048576,), data=C.data)
            for x_inner_init in range(32):
                C_1[x_outer * 32768 + x_inner_init * 1024 + y_outer * 32:x_outer * 32768 + x_inner_init * 1024 + y_outer * 32 + 32] = T.Broadcast(T.floa

[21:09:30] /home/patrick/Code/tvm/src/ir/transform.cc:440: Running pass tir.InjectPrefetch
[21:09:30] /home/patrick/Code/tvm/src/ir/transform.cc:440: Running pass tir.TextureFlatten
[21:09:30] /home/patrick/Code/tvm/src/ir/transform.cc:440: Running pass tir.StorageFlatten
[21:09:30] /home/patrick/Code/tvm/src/ir/transform.cc:440: Running pass tir.BufferShapeLegalize
[21:09:30] /home/patrick/Code/tvm/src/ir/transform.cc:440: Running pass tir.BufferStrideLegalize
[21:09:30] /home/patrick/Code/tvm/src/ir/transform.cc:440: Running pass tir.ThreadScopePropagate
[21:09:30] /home/patrick/Code/tvm/src/ir/transform.cc:440: Running pass tir.BufferBindUnwrapper
[21:09:30] /home/patrick/Code/tvm/src/ir/transform.cc:440: Running pass tir.ApplyLayoutTransforms
[21:09:30] /home/patrick/Code/tvm/src/ir/transform.cc:440: Running pass tir.StorageFlattener
[21:09:30] /home/patrick/Code/tvm/src/ir/transform.cc:440: Running pass tir.AssertSimplifier
[21:09:30] /home/patrick/Code/tvm/src/ir/transform.cc:440

通过缓存优化块写入

到目前为止，所有的优化都集中再有效地访问和计算来自A和B矩阵的数据，从而计算C矩阵，算子会逐块将结果写入C，访问模式不是顺序的，可用顺序缓存数组来解决这个问题，用cache_write、compute_at和unroll的组合来保存块结果，并在所有块结果准备好时写入C

In [10]:
s = te.create_schedule(C.op)

# Allocate write cache
# 分配写缓存
CC = s.cache_write(C, "global")

xo, yo, xi, yi = s[C].tile(C.op.axis[0], C.op.axis[1], bn, bn)

# Write cache is computed at yo
# 写缓存在 yo 处被计算
s[CC].compute_at(s[C], yo)

# New inner axes
# 新的内部轴
xc, yc = s[CC].op.axis

(k,) = s[CC].op.reduce_axis
ko, ki = s[CC].split(k, factor=4)
s[CC].reorder(ko, xc, ki, yc)
s[CC].unroll(ki)
s[CC].vectorize(yc)

x, y, z = s[packedB].op.axis
s[packedB].vectorize(z)
s[packedB].parallel(x)

evaluate_operation(s, [A, B, C], target=target, name="mmult", optimization="block caching", log=log)

# Here is the generated IR after write cache blocking.
# 写缓存块级化后生成的 IR。
print(tvm.lower(s, [A, B, C], simple_mode=True))

[21:13:14] /home/patrick/Code/tvm/src/ir/transform.cc:440: Running pass tir.InjectPrefetch
[21:13:14] /home/patrick/Code/tvm/src/ir/transform.cc:440: Running pass tir.TextureFlatten
[21:13:14] /home/patrick/Code/tvm/src/ir/transform.cc:440: Running pass tir.StorageFlatten
[21:13:14] /home/patrick/Code/tvm/src/ir/transform.cc:440: Running pass tir.BufferShapeLegalize
[21:13:14] /home/patrick/Code/tvm/src/ir/transform.cc:440: Running pass tir.BufferStrideLegalize
[21:13:14] /home/patrick/Code/tvm/src/ir/transform.cc:440: Running pass tir.ThreadScopePropagate
[21:13:14] /home/patrick/Code/tvm/src/ir/transform.cc:440: Running pass tir.BufferBindUnwrapper
[21:13:14] /home/patrick/Code/tvm/src/ir/transform.cc:440: Running pass tir.ApplyLayoutTransforms
[21:13:14] /home/patrick/Code/tvm/src/ir/transform.cc:440: Running pass tir.StorageFlattener
[21:13:14] /home/patrick/Code/tvm/src/ir/transform.cc:440: Running pass tir.AssertSimplifier
[21:13:14] /home/patrick/Code/tvm/src/ir/transform.cc:440

block caching: 0.059557
# from tvm.script import ir as I
# from tvm.script import tir as T

@I.ir_module
class Module:
    @T.prim_func
    def main(A: T.Buffer((1024, 1024), "float32"), B: T.Buffer((1024, 1024), "float32"), C: T.Buffer((1024, 1024), "float32")):
        T.func_attr({"from_legacy_te_schedule": True, "global_symbol": "main", "tir.noalias": True})
        packedB = T.allocate([32768], "float32x32", "global")
        C_global = T.allocate([1024], "float32", "global")
        packedB_1 = T.Buffer((32768,), "float32x32", data=packedB)
        for x in T.parallel(32):
            for y in range(1024):
                B_1 = T.Buffer((1048576,), data=B.data)
                packedB_1[x * 1024 + y] = B_1[y * 1024 + x * 32:y * 1024 + x * 32 + 32]
        for x_outer, y_outer in T.grid(32, 32):
            C_global_1 = T.Buffer((1024,), data=C_global)
            for x_c_init in range(32):
                C_global_1[x_c_init * 32:x_c_init * 32 + 32] = T.Broadcast(T.float32(0), 32

[21:13:15] /home/patrick/Code/tvm/src/ir/transform.cc:440: Running pass tir.InjectPrefetch
[21:13:15] /home/patrick/Code/tvm/src/ir/transform.cc:440: Running pass tir.TextureFlatten
[21:13:15] /home/patrick/Code/tvm/src/ir/transform.cc:440: Running pass tir.StorageFlatten
[21:13:15] /home/patrick/Code/tvm/src/ir/transform.cc:440: Running pass tir.BufferShapeLegalize
[21:13:15] /home/patrick/Code/tvm/src/ir/transform.cc:440: Running pass tir.BufferStrideLegalize
[21:13:15] /home/patrick/Code/tvm/src/ir/transform.cc:440: Running pass tir.ThreadScopePropagate
[21:13:15] /home/patrick/Code/tvm/src/ir/transform.cc:440: Running pass tir.BufferBindUnwrapper
[21:13:15] /home/patrick/Code/tvm/src/ir/transform.cc:440: Running pass tir.ApplyLayoutTransforms
[21:13:15] /home/patrick/Code/tvm/src/ir/transform.cc:440: Running pass tir.StorageFlattener
[21:13:15] /home/patrick/Code/tvm/src/ir/transform.cc:440: Running pass tir.AssertSimplifier
[21:13:15] /home/patrick/Code/tvm/src/ir/transform.cc:440

并行化

到目前为止，仅设计了用单核来计算。几乎所有现代处理器都有多个内核，计算可以从并行计算中受益。最后的优化将利用线程级并行（thread-level parallelization）

In [11]:
# parallel
# 并行化
s[C].parallel(xo)

x, y, z = s[packedB].op.axis
s[packedB].vectorize(z)
s[packedB].parallel(x)

evaluate_operation(
    s, [A, B, C], target=target, name="mmult", optimization="parallelization", log=log
)

# Here is the generated IR after parallelization.
# 并行化后生成的 IR。
print(tvm.lower(s, [A, B, C], simple_mode=True))

[21:14:11] /home/patrick/Code/tvm/src/ir/transform.cc:440: Running pass tir.InjectPrefetch
[21:14:11] /home/patrick/Code/tvm/src/ir/transform.cc:440: Running pass tir.TextureFlatten
[21:14:11] /home/patrick/Code/tvm/src/ir/transform.cc:440: Running pass tir.StorageFlatten
[21:14:11] /home/patrick/Code/tvm/src/ir/transform.cc:440: Running pass tir.BufferShapeLegalize
[21:14:11] /home/patrick/Code/tvm/src/ir/transform.cc:440: Running pass tir.BufferStrideLegalize
[21:14:11] /home/patrick/Code/tvm/src/ir/transform.cc:440: Running pass tir.ThreadScopePropagate
[21:14:11] /home/patrick/Code/tvm/src/ir/transform.cc:440: Running pass tir.BufferBindUnwrapper
[21:14:11] /home/patrick/Code/tvm/src/ir/transform.cc:440: Running pass tir.ApplyLayoutTransforms
[21:14:11] /home/patrick/Code/tvm/src/ir/transform.cc:440: Running pass tir.StorageFlattener
[21:14:11] /home/patrick/Code/tvm/src/ir/transform.cc:440: Running pass tir.AssertSimplifier
[21:14:11] /home/patrick/Code/tvm/src/ir/transform.cc:440

parallelization: 0.018325
# from tvm.script import ir as I
# from tvm.script import tir as T

@I.ir_module
class Module:
    @T.prim_func
    def main(A: T.Buffer((1024, 1024), "float32"), B: T.Buffer((1024, 1024), "float32"), C: T.Buffer((1024, 1024), "float32")):
        T.func_attr({"from_legacy_te_schedule": True, "global_symbol": "main", "tir.noalias": True})
        packedB = T.allocate([32768], "float32x32", "global")
        packedB_1 = T.Buffer((32768,), "float32x32", data=packedB)
        for x in T.parallel(32):
            for y in range(1024):
                B_1 = T.Buffer((1048576,), data=B.data)
                packedB_1[x * 1024 + y] = B_1[y * 1024 + x * 32:y * 1024 + x * 32 + 32]
        for x_outer in T.parallel(32):
            C_global = T.allocate([1024], "float32", "global")
            for y_outer in range(32):
                C_global_1 = T.Buffer((1024,), data=C_global)
                for x_c_init in range(32):
                    C_global_1[x_c_init * 32:x_c

[21:14:12] /home/patrick/Code/tvm/src/ir/transform.cc:440: Running pass tir.InjectPrefetch
[21:14:12] /home/patrick/Code/tvm/src/ir/transform.cc:440: Running pass tir.TextureFlatten
[21:14:12] /home/patrick/Code/tvm/src/ir/transform.cc:440: Running pass tir.StorageFlatten
[21:14:12] /home/patrick/Code/tvm/src/ir/transform.cc:440: Running pass tir.BufferShapeLegalize
[21:14:12] /home/patrick/Code/tvm/src/ir/transform.cc:440: Running pass tir.BufferStrideLegalize
[21:14:12] /home/patrick/Code/tvm/src/ir/transform.cc:440: Running pass tir.ThreadScopePropagate
[21:14:12] /home/patrick/Code/tvm/src/ir/transform.cc:440: Running pass tir.BufferBindUnwrapper
[21:14:12] /home/patrick/Code/tvm/src/ir/transform.cc:440: Running pass tir.ApplyLayoutTransforms
[21:14:12] /home/patrick/Code/tvm/src/ir/transform.cc:440: Running pass tir.StorageFlattener
[21:14:12] /home/patrick/Code/tvm/src/ir/transform.cc:440: Running pass tir.AssertSimplifier
[21:14:12] /home/patrick/Code/tvm/src/ir/transform.cc:440

矩阵乘法示例总结

生成代码开始接近带有数学内核库（MKL）的numpy的行呢个，由于一直在记录性能，下面简单比较结果

In [12]:
baseline = log[0][1]
print("%s\t%s\t%s" % ("Operator".rjust(20), "Timing".rjust(20), "Performance".rjust(20)))
for result in log:
    print(
        "%s\t%s\t%s"
        % (result[0].rjust(20), str(result[1]).rjust(20), str(result[1] / baseline).rjust(20))
    )

            Operator	              Timing	         Performance
                none	  2.5328502660999996	                 1.0
            blocking	         0.167352122	 0.06607264718323967
       vectorization	        0.1667185534	 0.06582250661690625
    loop permutation	 0.06669915099999998	0.026333633650875526
       array packing	        0.0631674183	0.024939262752891878
       block caching	0.059557385799999994	0.023513978144355338
     parallelization	        0.0183245433	0.007234751909837739
